# Elastic net (MADlib v1.10+)
Demonstrates elastic net, including grouping and cross validation, which were introduced in MADlib v1.10

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
%sql postgresql://gpdbchina@10.194.10.68:55000/madlib
#%sql postgresql://fmcquillan@localhost:5432/madlib
#%sql postgresql://gpadmin@54.197.30.46:10432/gpadmin

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-12-g6939fd6, cmake configuration time: Thu Dec 22 00:35:17 UTC 2016, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


## 1.  Create data set
House prices and characteristics.

In [4]:
%%sql 
DROP TABLE IF EXISTS houses;

CREATE TABLE houses ( id INT,
                      tax INT,
                      bedroom INT,
                      bath FLOAT,
                      price INT,
                      size INT,
                      lot INT,
                      zipcode INT);

INSERT INTO houses (id, tax, bedroom, bath, price, size, lot, zipcode) VALUES
(1  ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100  , 94301),
(2  , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000  , 94301),
(3  ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500  , 94301),
(4  ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500  , 94301),
(5  , 1320 ,       3 ,    2 , 133000 , 1500 , 30000  , 94301),
(6  , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700  , 94301),
(7  , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000  , 94301),
(8  ,  680 ,       2 ,    1 , 142500 , 1170 , 22000  , 94301),
(9  , 1840 ,       3 ,    2 , 160000 , 1500 , 19000  , 94301),
(10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000  , 94301),
(11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500  , 94301),
(12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000  , 94301),
(13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000  , 94301),
(14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000  , 94301),
(15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000  , 94301),
(16 ,  770 ,       2 ,    2 ,  91000 , 1300 , 17500  , 76010),
(17 , 1220 ,       3 ,    2 , 132300 , 1500 , 30000  , 76010),
(18 , 1150 ,       2 ,    1 ,  91100 ,  820 , 25700  , 76010),
(19 , 2690 ,       3 ,  2.5 , 260011 , 2130 , 25000  , 76010),
(20 ,  780 ,       2 ,    1 , 141800 , 1170 , 22000  , 76010),
(21 , 1910 ,       3 ,    2 , 160900 , 1500 , 19000  , 76010),
(22 , 3600 ,       4 ,    2 , 239000 , 2790 , 20000  , 76010),
(23 , 1600 ,       3 ,    1 ,  81010 , 1030 , 17500  , 76010),
(24 , 1590 ,       3 ,    2 , 117910 , 1250 , 20000  , 76010),
(25 , 3200 ,       3 ,    2 , 141100 , 1760 , 38000  , 76010),
(26 , 2270 ,       2 ,    3 , 148011 , 1550 , 14000  , 76010),
(27 ,  750 ,       3 ,  1.5 ,  66000 , 1450 , 12000  , 76010);

SELECT * FROM houses ORDER BY id;

Done.
Done.
27 rows affected.
27 rows affected.


id,tax,bedroom,bath,price,size,lot,zipcode
1,590,2,1.0,50000,770,22100,94301
2,1050,3,2.0,85000,1410,12000,94301
3,20,3,1.0,22500,1060,3500,94301
4,870,2,2.0,90000,1300,17500,94301
5,1320,3,2.0,133000,1500,30000,94301
6,1350,2,1.0,90500,820,25700,94301
7,2790,3,2.5,260000,2130,25000,94301
8,680,2,1.0,142500,1170,22000,94301
9,1840,3,2.0,160000,1500,19000,94301
10,3680,4,2.0,240000,2790,20000,94301


## 2. Train the model

In [16]:
%%sql
DROP TABLE IF EXISTS houses_en, houses_en_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en',               -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en;

Done.
1 rows affected.
1 rows affected.


family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[22.785125212, 10707.9542786, 54.796130396]","[22.785125212, 10707.9542786, 54.796130396]",-7798.79034492,-512248641.969,True,375


# 3. Prediction
Evaluate residuals.

In [17]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_all,              -- Coefficients
            m.intercept,             -- Intercept
            ARRAY[tax,bath,size]     -- Features (corresponding to coefficients)
            ) AS predict
    FROM houses, houses_en m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,58545.4082137,-8545.40821368
2,85000,114804.043543,-29804.0435432
3,22500,61448.7646577,-38948.7646577
4,90000,104675.146662,-14675.1466615
5,133000,125887.679086,7112.32091388
6,90500,78601.9098946,11898.0901054
7,260000,199257.352437,60742.6475635
8,142500,82514.5216412,59985.4783588
9,160000,137735.944196,22264.0558036
10,240000,250347.582797,-10347.5827973


# 4. Grouping 
Group on zip code.

In [18]:
%%sql
DROP TABLE IF EXISTS houses_en1, houses_en1_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en1',              -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 'zipcode',                 -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en1;

Done.
1 rows affected.
2 rows affected.


zipcode,family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
76010,gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[15.6325563808, 10166.6562969, 57.8690658309]","[15.6325563808, 10166.6562969, 57.8690658309]",513.875727396,-538806627.977,True,10000
94301,gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[27.0542060555, 12351.5246493, 47.5833358009]","[27.0542060555, 12351.5246493, 47.5833358009]",-7191.20234448,-519199958.999,True,10000


In [19]:
%%sql
SELECT madlib.elastic_net_predict(
                'houses_en1',             -- Model table
                'houses',                 -- New source data table
                'id',                     -- Unique ID associated with each row
                'houses_en1_prediction'   -- Table to store prediction result
              );

SELECT  houses.id,
        houses.price,
        houses_en1_prediction.prediction,
        houses.price - houses_en1_prediction.prediction AS residual
FROM houses_en1_prediction, houses
WHERE houses.id = houses_en1_prediction.id ORDER BY id;

1 rows affected.
27 rows affected.


id,price,prediction,residual
1,50000,57761.4724443,-7761.47244426
2,85000,113011.266792,-28011.2667917
3,22500,56139.7423749,-33639.7423749
4,90000,102907.342764,-12907.3427636
5,133000,124598.402649,8401.59735127
6,90500,80701.8358365,9798.16416352
7,260000,200521.34943,59478.6505705
8,142500,79229.6853096,63270.3146904
9,160000,138666.589798,21333.4102024
10,240000,249828.832123,-9828.83212287


## 5.  When coef_nonzero is different from coef_all
Train

In [20]:
%%sql
DROP TABLE IF EXISTS houses_en2, houses_en2_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en2',              -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 1,                         -- Alpha value
                                 30000,                     -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en2;

Done.
1 rows affected.
1 rows affected.


family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'size']","[6.94744249834, 29.7137297658]","[6.94744249834, 0.0, 29.7137297658]",74445.7039382,-1635348585.07,True,151


Prediction function with coef_all to evaluate residuals.

In [21]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_all,                   -- All coefficients
            m.intercept,                  -- Intercept
            ARRAY[tax,bath,size]          -- All features
            ) AS predict
    FROM houses, houses_en2 m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,101424.266932,-51424.2669319
2,85000,123636.877531,-38636.8775312
3,22500,106081.20634,-83581.2063399
4,90000,119117.827607,-29117.8276073
5,133000,128186.922685,4813.07731529
6,90500,108190.009719,-17690.0097189
7,260000,157119.31291,102880.68709
8,142500,113935.028663,28564.9713369
9,160000,131799.592784,28200.4072162
10,240000,182913.598379,57086.4016213


We can speed up the prediction function with coef_nonzero to evaluate residuals. This requires the user to examine the feature_selected column in the result table to construct the correct set of independent variables to provide to the prediction function.

In [22]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_nonzero,               -- Non-zero coefficients
            m.intercept,                  -- Intercept
            ARRAY[tax,size]               -- Features corresponding to non-zero coefficients
            ) AS predict
    FROM houses, houses_en2 m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,101424.266932,-51424.2669319
2,85000,123636.877531,-38636.8775312
3,22500,106081.20634,-83581.2063399
4,90000,119117.827607,-29117.8276073
5,133000,128186.922685,4813.07731529
6,90500,108190.009719,-17690.0097189
7,260000,157119.31291,102880.68709
8,142500,113935.028663,28564.9713369
9,160000,131799.592784,28200.4072162
10,240000,182913.598379,57086.4016213


## 6.  Cross validation
Here we use 3-fold cross validation with 3 automatically generated
lambda values and 3 specified alpha values.  (This can take some time
to run since elastic net is effectively being called 27 times.)

In [9]:
%%sql
DROP TABLE IF EXISTS houses_en3, houses_en3_summary, houses_en3_cv;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en3',               -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 $$ n_folds = 3,            -- Optimizer parameters
                                    validation_result=houses_en3_cv,
                                    n_lambdas = 3, 
                                    alpha = {0, 0.1, 1}
                                 $$,                       
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en3;

Done.
1 rows affected.
1 rows affected.


family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[22.4584783679, 11657.0825871, 52.1622899664]","[22.4584783679, 11657.0825871, 52.1622899664]",-5067.27288499,-543193170.15,True,392


Details of the cross validation:

In [13]:
%%sql
SELECT * FROM houses_en3_cv ORDER BY lambda_value DESC, alpha ASC;

9 rows affected.


std,alpha,lambda_value,mean
1.80536123195e+110,0.0,100000.0,-1.41777698585e+110
1.72846143163e+107,0.1,100000.0,-1.19953054719e+107
2485189261.38,1.0,100000.0,-4175743937.91
2424765457.66,0.0,100.0,-4054694238.18
2418294966.72,0.1,100.0,-4041768667.28
483327430.802,1.0,100.0,-1458791218.11
426795110.876,0.0,0.1,-1442293698.38
429680202.16,0.1,0.1,-1442705511.6
485107796.02,1.0,0.1,-1459206061.39
